# Contoso Chat Assistant

## Objetive
This notebook demonstrates the following:

- Function Calling: Call a function to get the customer info and previous orders
- Reading data from a file: Find products information by reading a csv file

Reference
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)



### Parameters

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

In [ ]:
import html
import io
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable
import json

import requests
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
# from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
# from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile

### Create an AzureOpenAI client

In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Get Customer Info

In [ ]:
def get_customer_info(id: str) -> str:
    get_customer_api = os.getenv("CUSTOMER_INFO_API")

    # ?id=1&partitionKeyValue=1
    response = requests.get(f"{get_customer_api}?id={id}&partitionKeyValue={id}")
    return json.dumps(response.json())

In [ ]:
# Test the function
print(get_customer_info("3"))

### Define the Assistant tools

In [ ]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_customer_info",
            "description": "Get Customer Information like full name and previous orders.",
            "parameters": {
                "type": "object",
                "properties": {"id": {"type": "integer", "description": "The customer id"}},
                "required": ["id"]
            }
        }
    }
]

### Upload the file(s)


In [ ]:
def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")
    

filePath = "../data/product_info/products.csv"
assistant_files = []
assistant_files.append(upload_file(client, filePath))

print(assistant_files)

In [ ]:
file_ids = [file.id for file in assistant_files]

### Create an Assistant and a Thread

In [ ]:
prompt_path = "customer_prompt.txt"
with open(prompt_path, "r") as f:
    instructions_txt = f.read()

assistant = client.beta.assistants.create(
    name="Contoso-Chat-Test",
    instructions=instructions_txt,
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

### Process Function calling

In [ ]:
def call_functions(client: AzureOpenAI, thread: Thread, run: Run) -> None:
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        if func_name == "get_customer_info":
            output = get_customer_info(id=arguments["id"])
            tool_outputs.append({"tool_call_id": action["id"], "output": output})
        else:
            raise ValueError(f"Unknown function: {func_name}")

    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

### Format and display the Assistant Messages for text and images

In [ ]:
def format_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            print(f"{message.role}:\n{item.text.value}\n")
            

### Process the user messages

In [ ]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=instructions_txt
    )

    print("processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            call_functions(client, thread, run)
        else:
            time.sleep(5)

### Have a conversation with the Assistant

In [ ]:
process_message("What can you tell me about your jackets?")

In [ ]:
process_message("What can you tell me about your jackets? Find products documentation in file products.csv. Use the columns 'name' and 'description' to find information about the product requested")

In [ ]:
process_message("What products do you have in file products.csv?")

In [ ]:
process_message("Can you remember me what products I have ordered before? My user id is 2")

In [ ]:
# if you want to start a new thread
client.beta.threads.delete(thread.id)
thread = client.beta.threads.create()